# Los Angeles MIDI Dataset Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os

import math
import statistics
import random
from collections import Counter
import shutil
import difflib
from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

if not os.path.exists('/content/Output'):
    os.makedirs('/content/Output')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD SOURCE MIDI DATASET)

In [ ]:
#@title Download original LAKH MIDI Dataset

%cd /content/Dataset/

!wget 'http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz'
!tar -xvf 'lmd_full.tar.gz'
!rm 'lmd_full.tar.gz'

%cd /content/

# (FILE LIST)

In [ ]:
#@title Save file list
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)

TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/filez')

In [ ]:
#@title Load file list
filez = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/filez')
print('Done!')

# (PROCESS)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

START_FILE_NUMBER = 0
LAST_SAVED_BATCH_COUNT = 0

input_files_count = START_FILE_NUMBER
files_count = LAST_SAVED_BATCH_COUNT

melody_chords_f = []

pitches_sums = []
pitches_counts = []

pitches_data = []

stats = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

print('Processing MIDI files. Please wait...')
print('=' * 70)

for f in tqdm(filez[START_FILE_NUMBER:]):
    try:
        input_files_count += 1

        fn = os.path.basename(f)
        fn1 = fn.split('.mid')[0]

        #=======================================================
        # START PROCESSING

        # Convering MIDI to ms score with MIDI.py module
        score = TMIDIX.midi2ms_score(open(f, 'rb').read())

        events_matrix = []
        itrack = 1
        
        while itrack < len(score):
          for event in score[itrack]:         
              if event[0] == 'note':
                  if event[3] == 9:
                    event[4] = (event[4] % 128)+128
                  else:
                    event[4] = (event[4] % 128)
                    
                  events_matrix.append(event)
          itrack += 1
        
        if len(events_matrix) >= 256:

            events_matrix.sort(key=lambda x: x[1])

            times = [e[1] for e in events_matrix]
            durs = [e[2] for e in events_matrix]

            if min(times) >= 0 and min(durs) >= 0:
                if len([k for k,v in Counter(times).items() if v>1]) != 0:
                    
                    pitches = [e[4] for e in events_matrix]
                    pitches_sum = sum(pitches)
                    pitches_number = len(pitches)

                    if pitches_sum not in pitches_sums:
                        pitches_count = sorted([[key, val] for key,val in Counter(pitches).most_common()], reverse=True)
                        #=======================================================
                        
                        if [y[1] for y in pitches_count] not in pitches_counts:

                            # Saving every 5000 processed files
                            if files_count % 50000 == 0:
                              dir_count = ((files_count // 50000)+1) * 50000
                              dir_count_str = str(dir_count).zfill(7)
                              copy_path = '/content/Output/'+dir_count_str
                              if not os.path.exists(copy_path):
                                  os.mkdir(copy_path)
                              print('SAVING !!!')
                              print('=' * 70)
                              print('Saving processed files...')
                              print('=' * 70)
                              print('Processed so far:', files_count, 'out of', input_files_count, '===', files_count / input_files_count, 'good files ratio')
                              print('=' * 70)

                            if files_count % 5000 == 0:
                              print('=' * 70)
                              print('Processed so far:', files_count, 'out of', input_files_count, '===', files_count / input_files_count, 'good files ratio')
                              print('=' * 70)
                              TMIDIX.Tegridy_Any_Pickle_File_Writer(pitches_data, '/content/Output/pitches_data')

                            shutil.copy2(f, copy_path+'/'+fn)

                            # Processed files counter
                            files_count += 1
                            
                            data = []
                            
                            data = [fn.split('.mid')[0], times[-1], len(set(times))]
                            for p in pitches_count:
                                data.extend(p)
                                                       
                            pitches_data.append(data)
                            
                            pitches_sums.append(pitches_sum)
                            pitches_counts.append([y[1] for y in pitches_count])


    except KeyboardInterrupt:
        print('Saving current progress and quitting...')
        break  

    except Exception as ex:
        print('WARNING !!!')
        print('=' * 70)
        print('Bad MIDI:', f)
        print('Error detected:', ex)
        print('=' * 70)
        continue

# Saving last processed files...
print('=' * 70)
print('Saving processed files...')
print('=' * 70)
print('Processed so far:', files_count, 'out of', input_files_count, '===', files_count / input_files_count, 'good files ratio')
print('=' * 70)
TMIDIX.Tegridy_Any_Pickle_File_Writer(pitches_data, '/content/Output/pitches_data')

# Displaying resulting processing stats...
print('=' * 70)
print('Done!')   
print('=' * 70)

print('Resulting Stats:')
print('=' * 70)
print('Total good processed MIDI files:', files_count)
print('=' * 70)

# Congrats! You did it! :)